## Rohan Bhatt
#### UID: 117942330

##### Research Question: Are bank failures more closely correlated with general economic recessions or with specific financial trends (e.g., Crypto boom, ML boom)?
##### This question was interesting to me because I have worked in the fintech space for several of my internships, and since I was 18 I've always had an interest in the stock market (I learned to trade, and subsequently fail/break-even trading options), so given my general interest in the space, seeing the fall of FTX, SVB, and more sparked my interest. Were there failures like FTX with an industry during the .COM boom? Or do banks usually sell out during recessions? 

##### - The population we will be working with is all banks in the United States, with a focus on those who have failed since October 1st, 2000.
##### - Variables: Dependent variable: Bank Failure (binary, fail or no fail). Independent variables are many, including economic indicators like GDP growth rate, unemployment. inflation, etc as well as financial trend indicators like capitalization of certain markets i.e. crypto, investment in industries like AI, etc. Potentially confounding variables include bank size (total assets), geographic location, and regulatory changes
##### - Hypothesis: Bank failures are more strongly correlated with general economic recessions than with specific financial trends, its just that financial trends have exacerbated the risk of failure during economic pitfalls/downturns.
##### - Data collection: The primary dataset is the FDIC failed bank list, economic data collected will be historical data on GDP growth rates, unemployment rates, and other relative economic indicators from places like the Federal Reserve, Census Bureau, etc. Financial trend data will also be important like historical data on crypto market capitalization, and capitalization of specific markets over a 20 year period (10/1/2000-2024)

In [1]:
# %pip install pyspark matplotlib #ONLY RUN ON COLAB

#ONLY RUN LOCALLY
import findspark
import os
from pyspark.sql import SparkSession
findspark.init()
os.environ['PYSPARK_SUBMIT_ARGS'] = '--verbose pyspark-shell'
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [2]:
import re
from pyspark.sql.functions import to_date, col, countDistinct, isnan, when, count, year, reduce, regexp_replace, col
import matplotlib.pyplot as plt
df = spark.read.csv("banklist.csv", header=True, inferSchema=True)

df.show(5)
df.printSchema()

+--------------------+-------------+------+-----+----------------------+-------------+-----+
|          Bank Name�|        City�|State�|Cert�|Acquiring Institution�|Closing Date�| Fund|
+--------------------+-------------+------+-----+----------------------+-------------+-----+
|Republic First Ba...| Philadelphia|    PA|27332|  Fulton Bank, Nati...|    26-Apr-24|10546|
|       Citizens Bank|     Sac City|    IA| 8758|  Iowa Trust & Savi...|     3-Nov-23|10545|
|Heartland Tri-Sta...|      Elkhart|    KS|25851|  Dream First Bank,...|    28-Jul-23|10544|
| First Republic Bank|San Francisco|    CA|59017|  JPMorgan Chase Ba...|     1-May-23|10543|
|      Signature Bank|     New York|    NY|57053|   Flagstar Bank, N.A.|    12-Mar-23|10540|
+--------------------+-------------+------+-----+----------------------+-------------+-----+
only showing top 5 rows

root
 |-- Bank Name�: string (nullable = true)
 |-- City�: string (nullable = true)
 |-- State�: string (nullable = true)
 |-- Cert�: inte

In [3]:
#Trying to get rid of weird symbols and cleaning up data
new_column_names = [re.sub(r'[^\x00-\x7F]', '', col_name) for col_name in df.columns]
df = df.toDF(*new_column_names)
df.show(5)

+--------------------+-------------+-----+-----+---------------------+------------+-----+
|           Bank Name|         City|State| Cert|Acquiring Institution|Closing Date| Fund|
+--------------------+-------------+-----+-----+---------------------+------------+-----+
|Republic First Ba...| Philadelphia|   PA|27332| Fulton Bank, Nati...|   26-Apr-24|10546|
|       Citizens Bank|     Sac City|   IA| 8758| Iowa Trust & Savi...|    3-Nov-23|10545|
|Heartland Tri-Sta...|      Elkhart|   KS|25851| Dream First Bank,...|   28-Jul-23|10544|
| First Republic Bank|San Francisco|   CA|59017| JPMorgan Chase Ba...|    1-May-23|10543|
|      Signature Bank|     New York|   NY|57053|  Flagstar Bank, N.A.|   12-Mar-23|10540|
+--------------------+-------------+-----+-----+---------------------+------------+-----+
only showing top 5 rows



### First steps will be to do time-series on the dates, and see commonalities (i.e. does anything hover more over 2008? from there will import more datasets)

## PART 2:
#### Preliminary info / looking for specific needs in data

In [4]:
print("Shape of data:")

print(f"Number of rows: {df.count()}")
print(f"Number of columns:  {len(df.columns)}")

print("Data Schema: ")
df.printSchema()

print("List of columns: ")
for column in df.columns: print(column)

Shape of data:
Number of rows: 569
Number of columns:  7
Data Schema: 
root
 |-- Bank Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Cert: integer (nullable = true)
 |-- Acquiring Institution: string (nullable = true)
 |-- Closing Date: string (nullable = true)
 |-- Fund: integer (nullable = true)

List of columns: 
Bank Name
City
State
Cert
Acquiring Institution
Closing Date
Fund


In [5]:
print("\n Descriptive Statistics")
df.describe().show()


 Descriptive Statistics
+-------+--------------------+-------+-----+------------------+---------------------+------------+------------------+
|summary|           Bank Name|   City|State|              Cert|Acquiring Institution|Closing Date|              Fund|
+-------+--------------------+-------+-----+------------------+---------------------+------------+------------------+
|  count|                 569|    569|  569|               569|                  569|         569|               569|
|   mean|                NULL|   NULL| NULL|31653.056239015816|                 NULL|        NULL|10042.210896309314|
| stddev|                NULL|   NULL| NULL|16464.868899588426|                 NULL|        NULL|1110.6370829809036|
|    min|1st American Stat...|Acworth|   AL|                91|      1st United Bank|    1-Aug-08|              4645|
|    max|               ebank|Wyoming|   WY|             59017|  Your Community Bank|    9-Sep-11|             10546|
+-------+--------------------+-

#### This makes sense; NULL is going to be shown for all the string variables, and there isn't necesssarily math involved in this. Although df.describe() gets the mean, stdev, etc, it doesn't make sense to find the mean and stdev for the cert and fund as those are just identifiers. Each variable count being 569 shows consistency; there aren't any missing values and its complete. There are 569 total number of bank failures. Below we can go analyze the number of unique entries per column:

In [6]:
print("\n looking for unique entries in columns")
for column in df.columns:
    unique_count = df.select(countDistinct(col(column))).collect()[0][0]
    print(f"{column}: {unique_count}")
print("----------------------------------")


 looking for unique entries in columns
Bank Name: 551
City: 436
State: 44
Cert: 569
Acquiring Institution: 303
Closing Date: 264
Fund: 569
----------------------------------


#### This is super useful as there were only 303 acquiring institutions, so it might be worth into looking later on the most common acquirer (i.e. JP Morgan, bigger Asian financial conglomerates). Below I am going to continue exploring this data set for any specific needs and other issues:



In [7]:
#doing type checks on the cert and fund columns to ensure type consistency

print("Checking for non-numeric values in cert & fund columns")

df.select(when(col("Cert").cast("int").isNull(), "non-numeric").otherwise("numeric").alias("Cert_check")).groupBy("Cert_check").count().show()
df.select(when(col("Fund").cast("int").isNull(), "non-numeric").otherwise("numeric").alias("Fund_check")).groupBy("Fund_check").count().show()

Checking for non-numeric values in cert & fund columns
+----------+-----+
|Cert_check|count|
+----------+-----+
|   numeric|  569|
+----------+-----+

+----------+-----+
|Fund_check|count|
+----------+-----+
|   numeric|  569|
+----------+-----+



In [8]:
#Analysis of bank failures over time
from pyspark.sql.functions import col, countDistinct, isnan, when, count, year, to_date, col, substring, concat, lit, avg
from pyspark.sql.types import *
print("\n Bank failures by year: ")
df.withColumn("Year", year("Closing Date")).groupBy("Year").count().orderBy("Year").show()

print("\nTop 10 states with most bank failures: ")
df.groupBy("State").count().orderBy(col("count").desc()).show(10)


 Bank failures by year: 
+----+-----+
|Year|count|
+----+-----+
|NULL|  569|
+----+-----+


Top 10 states with most bank failures: 
+-----+-----+
|State|count|
+-----+-----+
|   GA|   93|
|   FL|   76|
|   IL|   69|
|   CA|   43|
|   MN|   23|
|   WA|   19|
|   AZ|   16|
|   MO|   16|
|   MI|   14|
|   TX|   13|
+-----+-----+
only showing top 10 rows



In [9]:
#Trying to figure out why the year's arent showing properly:
df.select("Closing Date").show(10, False)

+------------+
|Closing Date|
+------------+
|26-Apr-24   |
|3-Nov-23    |
|28-Jul-23   |
|1-May-23    |
|12-Mar-23   |
|10-Mar-23   |
|23-Oct-20   |
|16-Oct-20   |
|3-Apr-20    |
|14-Feb-20   |
+------------+
only showing top 10 rows



In [10]:
df = df.withColumn('Closing Date', to_date(col('Closing Date'), 'd-MMM-yy'))
df.printSchema()

root
 |-- Bank Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Cert: integer (nullable = true)
 |-- Acquiring Institution: string (nullable = true)
 |-- Closing Date: date (nullable = true)
 |-- Fund: integer (nullable = true)



In [11]:
#now that we casted the variable to a date object, we can extract the year
df = df.withColumn('Year', year(col('Closing Date')))
print("\nBank failures by year:")
df.groupBy("Year").count().orderBy("Year").show()


Bank failures by year:
+----+-----+
|Year|count|
+----+-----+
|2000|    2|
|2001|    4|
|2002|   11|
|2003|    3|
|2004|    4|
|2007|    3|
|2008|   25|
|2009|  140|
|2010|  157|
|2011|   92|
|2012|   51|
|2013|   24|
|2014|   18|
|2015|    8|
|2016|    5|
|2017|    8|
|2019|    4|
|2020|    4|
|2023|    5|
|2024|    1|
+----+-----+



##### This shows the bank failures were relatively low 2000-2007 (minor bump in 2002), spike from 2008-2011 coinciding with the global financial crisis, and post 2012 decreased onwards. 

In [12]:
df.show(6, truncate=False)

+-------------------------------------+-------------+-----+-----+-----------------------------------+------------+-----+----+
|Bank Name                            |City         |State|Cert |Acquiring Institution              |Closing Date|Fund |Year|
+-------------------------------------+-------------+-----+-----+-----------------------------------+------------+-----+----+
|Republic First Bank dba Republic Bank|Philadelphia |PA   |27332|Fulton Bank, National Association  |2024-04-26  |10546|2024|
|Citizens Bank                        |Sac City     |IA   |8758 |Iowa Trust & Savings Bank          |2023-11-03  |10545|2023|
|Heartland Tri-State Bank             |Elkhart      |KS   |25851|Dream First Bank, N.A.             |2023-07-28  |10544|2023|
|First Republic Bank                  |San Francisco|CA   |59017|JPMorgan Chase Bank, N.A.          |2023-05-01  |10543|2023|
|Signature Bank                       |New York     |NY   |57053|Flagstar Bank, N.A.                |2023-03-12  |1054

In [13]:
#Manually pulling out the unclean row
df.filter(col('Cert') == 24735).show()

#defining a regex pattern to match non-ascii characters
non_ascii = r'[^\x00-\x7F]'
weird_rows = df.filter(col('Acquiring Institution').rlike(non_ascii)).show()

+-------------------+-----------+-----+-----+---------------------+------------+-----+----+
|          Bank Name|       City|State| Cert|Acquiring Institution|Closing Date| Fund|Year|
+-------------------+-----------+-----+-----+---------------------+------------+-----+----+
|Silicon Valley Bank|Santa Clara|   CA|24735| First�Citizens Ba...|  2023-03-10|10539|2023|
+-------------------+-----------+-----+-----+---------------------+------------+-----+----+

+-------------------+-----------+-----+-----+---------------------+------------+-----+----+
|          Bank Name|       City|State| Cert|Acquiring Institution|Closing Date| Fund|Year|
+-------------------+-----------+-----+-----+---------------------+------------+-----+----+
|Silicon Valley Bank|Santa Clara|   CA|24735| First�Citizens Ba...|  2023-03-10|10539|2023|
+-------------------+-----------+-----+-----+---------------------+------------+-----+----+



In [14]:
#Just to make sure all columns are clean:

columns_to_clean = ['Bank Name', 'City', 'State', 'Cert', 'Acquiring Institution', 'Closing Date', 'Fund']
for column_name in columns_to_clean:
    df = df.withColumn(
        column_name,
        regexp_replace(col(column_name), non_ascii, '-')
    )
df.filter(col("Bank Name") == 'Silicon Valley Bank').show(truncate=False)

+-------------------+-----------+-----+-----+-----------------------------------+------------+-----+----+
|Bank Name          |City       |State|Cert |Acquiring Institution              |Closing Date|Fund |Year|
+-------------------+-----------+-----+-----+-----------------------------------+------------+-----+----+
|Silicon Valley Bank|Santa Clara|CA   |24735|First-Citizens Bank & Trust Company|2023-03-10  |10539|2023|
+-------------------+-----------+-----+-----+-----------------------------------+------------+-----+----+



In [15]:
#I noticed first citizens bank doesn't have the "-" like the others, so lets fix that
df = df.withColumn(
    'Acquiring Institution',
    regexp_replace('Acquiring Institution', 'FirstCitizens', 'First-Citizens')
)
df.filter(col('Bank Name') == 'Silicon Valley Bank').show(truncate=False)

+-------------------+-----------+-----+-----+-----------------------------------+------------+-----+----+
|Bank Name          |City       |State|Cert |Acquiring Institution              |Closing Date|Fund |Year|
+-------------------+-----------+-----+-----+-----------------------------------+------------+-----+----+
|Silicon Valley Bank|Santa Clara|CA   |24735|First-Citizens Bank & Trust Company|2023-03-10  |10539|2023|
+-------------------+-----------+-----+-----+-----------------------------------+------------+-----+----+



**Finding specific data needs:**

As of this block I have cleaned the FDIC Failed Bank List. So far I have discussed conducting time-series analysis on the various financial institutions being acquired, and seeing what time period was common and from the 303 acquiring institutions out of the 569 total entries, seeing what (if any) specific institution was more common than the others and figuring it out why. Now that I have the bank failure list, I am going to add on datasets that represent general economic indicators and specific financial trends. I am going to add on a GDP growth rate dataset along with some Cryptocurrency market capitalization data so we can compare the failures to different economic periods and crypto trends.

#### Getting Bitcoin data from CoinGecko since 2013. I chose Bitcoin because it has always been the largest cryptocurrency and highly reflective of the market. Also as a student this data is not readily available, and the actual total market cap of all of crypto value is locked behind a paywall of $1000 a month (https://coinmarketcap.com/crypto-heatmap/). I also got the gdp data from https://www.bea.gov/data/gdp/gross-domestic-product

#### Let's check the overall schema for both datasets (GDP and crypto)

In [16]:
from pyspark.sql.functions import to_date, coalesce, avg
gdp_df = spark.read.csv('gdp.csv', header=True, inferSchema=True)
gdp_df.show(5)
gdp_df.printSchema()

btc_df = spark.read.csv('btc-usd-max.csv', header=True, inferSchema=True)
btc_df.show(5)
btc_df.printSchema()

+------+----------------------------------+---------------------------------------+
|   _c0|GDP in billions of current dollars|GDP in billions of chained 2017 dollars|
+------+----------------------------------+---------------------------------------+
|2000Q1|                               4.2|                                    1.5|
|2000Q2|                              10.2|                                    7.5|
|2000Q3|                               2.8|                                    0.4|
|2000Q4|                               4.6|                                    2.4|
|2001Q1|                               1.3|                                   -1.3|
+------+----------------------------------+---------------------------------------+
only showing top 5 rows

root
 |-- _c0: string (nullable = true)
 |-- GDP in billions of current dollars: double (nullable = true)
 |-- GDP in billions of chained 2017 dollars: double (nullable = true)

+-------------------+------+-------------

Data cleaning & preprocessing of the GDP dataset

In [17]:
#renaming this column to something referencable
gdp_df = gdp_df.withColumnRenamed('GDP in billions of current dollars', 'GDP_Current_Dollars') \
               .withColumnRenamed('GDP in billions of chained 2017 dollars', 'GDP_Chained_2017_Dollars') \
               .withColumnRenamed('_c0', 'Date')

#creating a date-time object to represent the first day of the quarter

gdp_df = gdp_df.withColumn('Year', substring('Date', 1, 4).cast('int'))
gdp_df = gdp_df.withColumn('Quarter', substring('Date', 6, 1).cast('int'))
#line below converts the string to a date type, concatenates the year with a hyphen, then gets the quarter by doing * 3 - 2 so i.e. Q4 is 4 * 3 - 2 so 
#its october or 10 which makes sense, then the day 01, and then the format of yyyy-M-d 
gdp_df = gdp_df.withColumn('Quarter_Start_Date', to_date(concat(col('Year'), lit('-'), (col('Quarter') * 3 - 2), lit('-01')), 'yyyy-M-d'))

gdp_df.select('Date', 'Year', 'Quarter', 'Quarter_Start_Date').show(5)
print("----------------------------")
gdp_df.show(15)

+------+----+-------+------------------+
|  Date|Year|Quarter|Quarter_Start_Date|
+------+----+-------+------------------+
|2000Q1|2000|      1|        2000-01-01|
|2000Q2|2000|      2|        2000-04-01|
|2000Q3|2000|      3|        2000-07-01|
|2000Q4|2000|      4|        2000-10-01|
|2001Q1|2001|      1|        2001-01-01|
+------+----+-------+------------------+
only showing top 5 rows

----------------------------
+------+-------------------+------------------------+----+-------+------------------+
|  Date|GDP_Current_Dollars|GDP_Chained_2017_Dollars|Year|Quarter|Quarter_Start_Date|
+------+-------------------+------------------------+----+-------+------------------+
|2000Q1|                4.2|                     1.5|2000|      1|        2000-01-01|
|2000Q2|               10.2|                     7.5|2000|      2|        2000-04-01|
|2000Q3|                2.8|                     0.4|2000|      3|        2000-07-01|
|2000Q4|                4.6|                     2.4|2000|   

Lets get the annual GDP growth rate by averaging the quarterly growth rates

In [18]:
annual_gdp_df = gdp_df.groupBy('Year').agg(avg('GDP_Current_Dollars').alias('Annual_GDP_Growth'))
annual_gdp_df = annual_gdp_df.orderBy('Year')
annual_gdp_df.show()

+----+-------------------+
|Year|  Annual_GDP_Growth|
+----+-------------------+
|2000|  5.449999999999999|
|2001|              2.175|
|2002|               3.75|
|2003|               6.45|
|2004|                6.4|
|2005|              6.375|
|2006|              5.375|
|2007|                4.8|
|2008|-0.6249999999999998|
|2009| 0.3500000000000003|
|2010|                4.5|
|2011| 3.4999999999999996|
|2012| 3.6500000000000004|
|2013|                4.7|
|2014|              4.225|
|2015|              2.925|
|2016|               3.55|
|2017|              4.975|
|2018|                4.4|
|2019|               4.85|
+----+-------------------+
only showing top 20 rows



Data cleaning and preprocessing of the bitcoin dataset

In [19]:
btc_df = btc_df.withColumn('Date', to_date(col('snapped_at'), 'yyyy-MM-dd HH:mm:ss z'))
btc_df = btc_df.drop('snapped_at')
btc_df.printSchema()

root
 |-- price: double (nullable = true)
 |-- market_cap: double (nullable = true)
 |-- total_volume: double (nullable = true)
 |-- Date: date (nullable = true)



In [20]:
from pyspark.sql.functions import month
btc_df = btc_df.withColumn('Year', year(col('Date')))
btc_df = btc_df.withColumn('Month', month(col('Date')))
btc_df.show(5)

+------+-------------+------------+----------+----+-----+
| price|   market_cap|total_volume|      Date|Year|Month|
+------+-------------+------------+----------+----+-----+
| 135.3| 1.50051759E9|         0.0|2013-04-27|2013|    4|
|141.96|1.575032004E9|         0.0|2013-04-28|2013|    4|
| 135.3|1.501657493E9|         0.0|2013-04-29|2013|    4|
| 117.0| 1.29895155E9|         0.0|2013-04-30|2013|    4|
|103.43|1.148667722E9|         0.0|2013-05-01|2013|    5|
+------+-------------+------------+----------+----+-----+
only showing top 5 rows



In [21]:
#Checking for missing values in market cap
btc_df.select([count(when(col(c).isNull(), c)).alias(c) for c in btc_df.columns]).show()

+-----+----------+------------+----+----+-----+
|price|market_cap|total_volume|Date|Year|Month|
+-----+----------+------------+----+----+-----+
|    0|         1|           0|   0|   0|    0|
+-----+----------+------------+----+----+-----+



In [22]:
btc_df = btc_df.dropna(subset=['market_cap'])
btc_df.select([count(when(col(c).isNull(), c)).alias(c) for c in btc_df.columns]).show()

+-----+----------+------------+----+----+-----+
|price|market_cap|total_volume|Date|Year|Month|
+-----+----------+------------+----+----+-----+
|    0|         0|           0|   0|   0|    0|
+-----+----------+------------+----+----+-----+



In [23]:
annual_btc_df = btc_df.groupBy('Year').agg(avg('market_cap').alias('Annual_Market_Cap'))
annual_btc_df = annual_btc_df.orderBy('Year')
annual_btc_df.show()

+----+--------------------+
|Year|   Annual_Market_Cap|
+----+--------------------+
|2013| 3.077474372939516E9|
|2014| 6.756528320092427E9|
|2015| 3.925042751283574E9|
|2016| 8.937041445128815E9|
|2017|6.746620332320473...|
|2018|1.291900764703961E11|
|2019|1.315301910601856...|
|2020|2.048368551120182E11|
|2021|8.908025690535553E11|
|2022|5.375604359614069E11|
|2023| 5.60908038513991E11|
|2024|1.184769581052524E12|
+----+--------------------+



In [42]:
# btc_df = btc_df.filter(col('market_cap') > 0) #removing non 0 entries

### Merging datasets with bank failure data

In [24]:
bank_failures_per_year = df.groupBy('Year').count().orderBy('Year').withColumnRenamed('count', 'Num_Bank_Failures')
bank_failures_per_year.show()

+----+-----------------+
|Year|Num_Bank_Failures|
+----+-----------------+
|2000|                2|
|2001|                4|
|2002|               11|
|2003|                3|
|2004|                4|
|2007|                3|
|2008|               25|
|2009|              140|
|2010|              157|
|2011|               92|
|2012|               51|
|2013|               24|
|2014|               18|
|2015|                8|
|2016|                5|
|2017|                8|
|2019|                4|
|2020|                4|
|2023|                5|
|2024|                1|
+----+-----------------+



In [25]:
bank_gdp_df = bank_failures_per_year.join(annual_gdp_df, on='Year', how='left')
bank_gdp_df.show()

+----+-----------------+-------------------+
|Year|Num_Bank_Failures|  Annual_GDP_Growth|
+----+-----------------+-------------------+
|2003|                3|               6.45|
|2007|                3|                4.8|
|2015|                8|              2.925|
|2023|                5|               5.85|
|2013|               24|                4.7|
|2014|               18|              4.225|
|2019|                4|               4.85|
|2004|                4|                6.4|
|2020|                4|  3.624999999999999|
|2012|               51| 3.6500000000000004|
|2009|              140| 0.3500000000000003|
|2016|                5|               3.55|
|2001|                4|              2.175|
|2024|                1|               5.15|
|2000|                2|  5.449999999999999|
|2010|              157|                4.5|
|2011|               92| 3.4999999999999996|
|2008|               25|-0.6249999999999998|
|2017|                8|              4.975|
|2002|    

In [26]:
bank_gdp_btc_df = bank_gdp_df.join(annual_btc_df, on='Year', how='left')
bank_gdp_btc_df = bank_gdp_btc_df.fillna({'Annual_Market_Cap': 0})
bank_gdp_btc_df.show()

+----+-----------------+-------------------+--------------------+
|Year|Num_Bank_Failures|  Annual_GDP_Growth|   Annual_Market_Cap|
+----+-----------------+-------------------+--------------------+
|2003|                3|               6.45|                 0.0|
|2007|                3|                4.8|                 0.0|
|2015|                8|              2.925| 3.925042751283574E9|
|2023|                5|               5.85| 5.60908038513991E11|
|2013|               24|                4.7| 3.077474372939516E9|
|2014|               18|              4.225| 6.756528320092427E9|
|2019|                4|               4.85|1.315301910601856...|
|2004|                4|                6.4|                 0.0|
|2020|                4|  3.624999999999999|2.048368551120182E11|
|2012|               51| 3.6500000000000004|                 0.0|
|2009|              140| 0.3500000000000003|                 0.0|
|2016|                5|               3.55| 8.937041445128815E9|
|2001|    

### 4. Looking for potential issues in dataset

In [27]:
#checking to make sure there are no missing years in the GDP data, and verifying the output is numeric
expected_years = range(bank_failures_per_year.agg({"Year": "min"}).collect()[0][0],
                       bank_failures_per_year.agg({"Year": "max"}).collect()[0][0] + 1)
missing_years = set(expected_years) - set([row['Year'] for row in annual_gdp_df.collect()])
print(f"Missing years in GDP data: {missing_years}")

Missing years in GDP data: set()


#### In terms of potential issues in the Bitcoin data, the data starts from 2013, all years prior to 2013 the annual market cap for BTC is set to zero. We already checked the original FDIC failed bank list for any missing or null values. 

##### For the FDIC list: 
* We cleared up the random "?" symbols, 
* found the number of unique entries matched the number of total entries through verifying via the cert, 
* checked for non-numeric values in cert & fund columns and found nothing wrong, 
* then casted the 'Closing Date' variable to be a date so we could sort different events by dates, 
* finally, to ensure each and every entry of every row/column was valid, we sorted through and identified any entry/cell that had a non-ascii character and fixed it via locating where it was (because we had already done so much prior pre-processing, only a couple data points had this issue), and adjusting those non-ascii entries to be proper (i.e. citizens-bank)
* besides cleaning the data, there only true variables you could quantify to look at for outliers are cert & fund, which would be incorrect to calculate a z-score for because those variables are identifiers for the acquisition being seen, it doesn't necessarily mean the entry is invalid.

#### Identifying potential issues in GDP data:

In [28]:
gdp_df.show(5)
gdp_df.printSchema()

# total rows
total_rows = gdp_df.count()
# distinct rows
distinct_rows = gdp_df.dropDuplicates().count()
# duplicates
duplicate_rows = total_rows - distinct_rows
print(f"Total rows: {total_rows}")
print(f"Distinct rows: {distinct_rows}")
print(f"Duplicate rows: {duplicate_rows}")

+------+-------------------+------------------------+----+-------+------------------+
|  Date|GDP_Current_Dollars|GDP_Chained_2017_Dollars|Year|Quarter|Quarter_Start_Date|
+------+-------------------+------------------------+----+-------+------------------+
|2000Q1|                4.2|                     1.5|2000|      1|        2000-01-01|
|2000Q2|               10.2|                     7.5|2000|      2|        2000-04-01|
|2000Q3|                2.8|                     0.4|2000|      3|        2000-07-01|
|2000Q4|                4.6|                     2.4|2000|      4|        2000-10-01|
|2001Q1|                1.3|                    -1.3|2001|      1|        2001-01-01|
+------+-------------------+------------------------+----+-------+------------------+
only showing top 5 rows

root
 |-- Date: string (nullable = true)
 |-- GDP_Current_Dollars: double (nullable = true)
 |-- GDP_Chained_2017_Dollars: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Quarter: in

In [29]:
from pyspark.sql.functions import col, when, count

#check for nulls in each column
null_counts = gdp_df.select([count(when(col(c).isNull(), c)).alias(c) for c in gdp_df.columns])
null_counts.show()

+----+-------------------+------------------------+----+-------+------------------+
|Date|GDP_Current_Dollars|GDP_Chained_2017_Dollars|Year|Quarter|Quarter_Start_Date|
+----+-------------------+------------------------+----+-------+------------------+
|   0|                  0|                       0|   0|      0|                 0|
+----+-------------------+------------------------+----+-------+------------------+



In [30]:
#checking for type inconsistencies
gdp_df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- GDP_Current_Dollars: double (nullable = true)
 |-- GDP_Chained_2017_Dollars: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Quarter_Start_Date: date (nullable = true)



We already converted the date column to a proper datetype:

In [32]:
# +------+----+-------+------------------+
# |  Date|Year|Quarter|Quarter_Start_Date|
# +------+----+-------+------------------+
# |2000Q1|2000|      1|        2000-01-01|
# |2000Q2|2000|      2|        2000-04-01|
# |2000Q3|2000|      3|        2000-07-01|
# |2000Q4|2000|      4|        2000-10-01|
# |2001Q1|2001|      1|        2001-01-01|
# +------+----+-------+------------------+
# only showing top 5 rows

# ----------------------------
# +------+-------------------+------------------------+----+-------+------------------+
# |  Date|GDP_Current_Dollars|GDP_Chained_2017_Dollars|Year|Quarter|Quarter_Start_Date|
# +------+-------------------+------------------------+----+-------+------------------+
# |2000Q1|                4.2|                     1.5|2000|      1|        2000-01-01|
# |2000Q2|               10.2|                     7.5|2000|      2|        2000-04-01|
# |2000Q3|                2.8|                     0.4|2000|      3|        2000-07-01|
# |2000Q4|                4.6|                     2.4|2000|      4|        2000-10-01|
# |2001Q1|                1.3|                    -1.3|2001|      1|        2001-01-01|
# |2001Q2|                5.0|                     2.5|2001|      2|        2001-04-01|
# |2001Q3|                0.0|                    -1.6|2001|      3|        2001-07-01|
# |2001Q4|                2.4|                     1.1|2001|      4|        2001-10-01|
# |2002Q1|                4.7|                     3.4|2002|      1|        2002-01-01|
# |2002Q2|                3.9|                     2.5|2002|      2|        2002-04-01|
# |2002Q3|                3.6|                     1.6|2002|      3|        2002-07-01|
# |2002Q4|                2.8|                     0.5|2002|      4|        2002-10-01|
# |2003Q1|                4.1|                     2.1|2003|      1|        2003-01-01|
# |2003Q2|                5.1|                     3.6|2003|      2|        2003-04-01|
# |2003Q3|                9.3|                     6.8|2003|      3|        2003-07-01|
# +------+-------------------+------------------------+----+-------+------------------+
# only showing top 15 rows

In [34]:
#Checking for outliers in GDP growth rate
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col

# mean and standard deviation
mean_val = gdp_df.select(_mean(col('GDP_Current_Dollars'))).collect()[0][0]
stddev_val = gdp_df.select(_stddev(col('GDP_Current_Dollars'))).collect()[0][0]

# adding Z-score column
gdp_df = gdp_df.withColumn('GDP_Z_Score', (col('GDP_Current_Dollars') - mean_val) / stddev_val)

# identifying outliers (Z-score > 3 or < -3)
gdp_outliers = gdp_df.filter((col('GDP_Z_Score') > 3) | (col('GDP_Z_Score') < -3))
gdp_outliers.select('Date', 'GDP_Current_Dollars', 'GDP_Z_Score').show()

+------+-------------------+------------------+
|  Date|GDP_Current_Dollars|       GDP_Z_Score|
+------+-------------------+------------------+
|2020Q2|              -29.1|-5.711285067478292|
|2020Q3|               40.0| 5.980628297481198|
+------+-------------------+------------------+



#### Although these are outliers, this makes sense and the entries should stay because this was clearly during the COVID-19 pandemic hence the intense volatility

### Analyzing the bitcoin dataset for potential issues

In [35]:
btc_df.printSchema()
# total rows
total_rows = btc_df.count()
# distinct rows
distinct_rows = btc_df.dropDuplicates().count()
# calculating duplicates
duplicate_rows = total_rows - distinct_rows
print(f"Total rows: {total_rows}")
print(f"Distinct rows: {distinct_rows}")
print(f"Duplicate rows: {duplicate_rows}")

root
 |-- price: double (nullable = true)
 |-- market_cap: double (nullable = true)
 |-- total_volume: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)

Total rows: 4180
Distinct rows: 4180
Duplicate rows: 0


In [36]:
# above shows no type inconsistencies, date is date type
# checking for nulls in each column
null_counts = btc_df.select([count(when(col(c).isNull(), c)).alias(c) for c in btc_df.columns])
null_counts.show()

+-----+----------+------------+----+----+-----+
|price|market_cap|total_volume|Date|Year|Month|
+-----+----------+------------+----+----+-----+
|    0|         0|           0|   0|   0|    0|
+-----+----------+------------+----+----+-----+



#### So because bitcoin is so volatile and has had exponential growth, so we can do log transformation before calculating z-scores to normalize the data a bit

In [39]:
from pyspark.sql.functions import log

# applying log to market cap
btc_df = btc_df.withColumn('log_market_cap', log(col('market_cap')))

# getting mean and stdev
mean_log_mc = btc_df.select(_mean(col('log_market_cap'))).collect()[0][0]
stddev_log_mc = btc_df.select(_stddev(col('log_market_cap'))).collect()[0][0]

# adding Z-score column
btc_df = btc_df.withColumn('Market_Cap_Z_Score', (col('log_market_cap') - mean_log_mc) / stddev_log_mc)

# seeing any outliers
btc_outliers = btc_df.filter((col('Market_Cap_Z_Score') > 3) | (col('Market_Cap_Z_Score') < -3))
btc_outliers.select('Date', 'market_cap', 'Market_Cap_Z_Score').show()

print("---------------------------------seeing z score without logging-------------------------")
mean_log_mc = btc_df.select(_mean(col('market_cap'))).collect()[0][0]
stddev_log_mc = btc_df.select(_stddev(col('market_cap'))).collect()[0][0]
btc_df = btc_df.withColumn('Market_Cap_Z_Score_no_log', (col('market_cap') - mean_log_mc) / stddev_log_mc)
btc_outliers = btc_df.filter((col('Market_Cap_Z_Score_no_log') > 3) | (col('Market_Cap_Z_Score_no_log') < -3))
btc_outliers.select('Date', 'market_cap', 'Market_Cap_Z_Score_no_log').show()

+----+----------+------------------+
|Date|market_cap|Market_Cap_Z_Score|
+----+----------+------------------+
+----+----------+------------------+

---------------------------------seeing z score without logging-------------------------
+----+----------+-------------------------+
|Date|market_cap|Market_Cap_Z_Score_no_log|
+----+----------+-------------------------+
+----+----------+-------------------------+



#### Summary of GDP & bitcoin data:
* Duplicates: No duplicate rows were found in the datasets
* Missing Values: No missing or null values, for btc no missing values in the 'market_cap' column were found except for initial days where 'total_volume' is zero, which we are not using in our analysis 
* Type Consistency: all columns are correctly typed
* Outliers: the outliers in the GDP set were identified using Z-scores which correspond to significant economic events like 2008 crisis so I kept those data points as they are crucial for our analysis later.
* Date Parsing: All dates have been successfully parsed into proper date formats, covering all quarters from 2000 to the latest available data.
* Conclusion: both datasets, after heavy pre-processing and cleaning are now ready for analysis.


#### 5. Any reorganization needed? No. All variables have valid variable names

### Detailed plan (ALREADY DONE IN PREVIOUS STEPS)
* GDP market cap: |Date|GDP_Current_Dollars|GDP_Chained_2017_Dollars|Year|Quarter|Quarter_Start_Date|
* BTC: |price|market_cap|total_volume|Date|Year|Month|
* FDIC: |summary|Bank Name|City|State|Cert|Acquiring Institution|Closing Date|Fund|

FDIC Dataset Cleaning Plan:

Issue: Non-ASCII characters in column names and entries
* Action: use regex to remove/replace non-ASCII characters
* Technique: use regexp_replace() function to clean data
* Why: makes sure data is consistent and prevents encoding issues

Issue: bad parsing of 'Closing Date' leading to null values
* Action: fix the date parsing format and adjust years 
* Technique: Use to_date() with the correct format and adjust two-digit year
* Why: date parsing is important for time-based analysis

Issue: duplicate entries
* Action: Check for and remove duplicate rows
* Technique: Use dropDuplicates() function
* Why: gets rid of redundant data that could skew analysis

GDP Dataset Cleaning Plan:

Issue: missing/null values
* Action: check if missing values exist and decide whether to include/exclude them
* Technique: Use dropna() or fillna() as appropriate
* Why: makes sure data is complete and with integrity
Issue: outliers in GDP growth rates
* Action: find and assess outliers to determine if they reflect actual economic events
* Technique: get Z-scores and use domain knowledge for interpretation (i.e. seing high gdp outlier with 08)
* Why: ensures validity of the dataset by keeping significant data points

Bitcoin Dataset Cleaning Plan:

Issue: exponential growth causing skewness in market cap data
* Action: apply log transformation to normalize data
* Technique: Use log() function on 'Market_Cap_USD'
* Why: normalizes data for better analysis
Issue: missing dates or zero market cap values.
* Action: check and handle missing or zero values appropriately
* Technique: Use explanatory notes (i.e. dates before 2013 are 0 because we dont have them/bitcoin didnt exist)


In [43]:
gdp_df.write.csv('cleaned_gdp.csv', header=True, mode='overwrite')
btc_df.write.csv('cleaned_btc.csv', header=True, mode='overwrite')
df.write.csv('cleaned_bank.csv', header=True, mode='overwrite')

Py4JJavaError: An error occurred while calling o695.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java: